<a href="https://colab.research.google.com/github/karaage0703/karaage-ai-book/blob/master/ch02/02_karaage_ai_book_image_classification_on_camera_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# カメラでの画像分類


学習させたモデルを使ってカメラ画像に対して画像分類をします。

注：このノートブックはハードウェアアクセラレータをGPUよりNone(CPU)で実行した方がレスポンスが良い場合もあります。

## 事前準備

In [22]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.11.0


In [23]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import time

モデルとラベルをGoogle Driveからコピーします。

In [24]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [25]:
!cp '/content/drive/MyDrive/Work/Class/AIR/2023-ai-lesson/my_model.h5' 'my_model.h5'
!cp '/content/drive/MyDrive/Work/Class/AIR/2023-ai-lesson/image_classification_model.h5' 'image_classification_model.h5'
!cp '/content/drive/MyDrive/Work/Class/AIR//2023-ai-lesson/labels.txt' 'labels.txt'

In [26]:
!ls -lh

total 107M
-rw-r--r-- 1 root root 3.1K Mar 11 08:10 colab_camera.py
drwx------ 6 root root 4.0K Mar 11 08:41 drive
-rw------- 1 root root  16M Mar 11 08:41 image_classification_model.h5
-rw------- 1 root root   12 Mar 11 08:41 labels.txt
-rw------- 1 root root  91M Mar 11 08:41 my_model.h5
drwxr-xr-x 2 root root 4.0K Mar 11 08:10 __pycache__
drwxr-xr-x 1 root root 4.0K Mar  9 18:58 sample_data


## AIモデルとラベルの読み込み

EfficientNetモデルを読み込む場合は以下実行ください

In [27]:
import tensorflow_hub as hub
keras_model = tf.keras.models.load_model('image_classification_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})
keras_model.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              4049564   
                                                                 
 dense_3 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 4,053,407
Trainable params: 3,843
Non-trainable params: 4,049,564
_________________________________________________________________


参考： https://github.com/tensorflow/tensorflow/issues/26835

データ水増しして学習させたMNISTベースの自作モデルを読み込む場合は、以下実行ください。
（EfficientNetを試したい場合は、こちらはスキップしてください）。

In [16]:
keras_model = tf.keras.models.load_model("my_model.h5")
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 activation (Activation)     (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 62, 62, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 64)        0         
                                                        

ラベルを読み込みます

In [28]:
labels = []
with open('labels.txt','r') as f:
  for line in f:
    labels.append(line.rstrip())
print(labels)

NUM_CLASSES = len(labels)

['choki', 'gu', 'pa']


## 画像分類デモの実行

Webカメラを使うJavascriptのコードをダウンロードします。

In [29]:
!wget https://raw.githubusercontent.com/fu11ji26/janken_dataset/master/colab_camera.py

--2023-03-11 08:42:54--  https://raw.githubusercontent.com/fu11ji26/janken_dataset/master/colab_camera.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3112 (3.0K) [text/plain]
Saving to: ‘colab_camera.py.1’

colab_camera.py.1   100%[===================>]   3.04K  --.-KB/s    in 0s      

2023-03-11 08:42:54 (52.9 MB/s) - ‘colab_camera.py.1’ saved [3112/3112]



コードを読み込みます。

In [30]:
import colab_camera

処理を行うコールバック関数を定義します。

In [31]:
import IPython
from google.colab import output
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64

def run(img_str):
  #decode to image
  decimg = base64.b64decode(img_str.split(',')[1], validate=True)
  decimg = Image.open(BytesIO(decimg))
  decimg = np.array(decimg, dtype=np.uint8); 
  decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

  # image classify
  img = cv2.resize(decimg, (64, 64))
  img = img_to_array(img)
  img = img / 255.0
  img = img[np.newaxis]
  img = np.asarray(img)
  preds = keras_model.predict(img)
  result = np.argmax(preds[0])

  # draw output
  out_img = decimg
  cv2.putText(out_img, labels[result], (10,50), \
      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)

  #encode to string
  _, encimg = cv2.imencode(".jpg", out_img,
      [int(cv2.IMWRITE_JPEG_QUALITY), 80])
  img_str = encimg.tobytes()
  img_str = ("data:image/jpeg;base64," +
      base64.b64encode(img_str).decode('utf-8'))
  return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)

デモを実施します。

In [32]:
colab_camera.use_cam()

<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 30ms/step


# 参考リンク

- https://qiita.com/a2kiti/items/f32de4f51a31d609e5a5